# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,-5.69,73,11,2.16,GS,1689488233
1,1,margaret river,-33.9500,115.0667,16.18,60,7,8.32,AU,1689488233
2,2,port-aux-francais,-49.3500,70.2167,1.60,59,90,14.48,TF,1689488233
3,3,daugavgriva,57.0432,24.0361,22.15,53,0,4.12,LV,1689488233
4,4,acarau,-2.8856,-40.1200,24.61,89,8,6.38,BR,1689488233


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points('Lng', 'Lat', 
                                      geo = True, 
                                      tiles = 'OSM', 
                                      color = 'Humidity', 
                                      size = 'Humidity',
                                      cmap = 'spring',
                                      hover_cols = ['City', 'Humidity'], 
                                      frame_height = 400, 
                                      frame_width = 750)

# Display the map
map_plot_humidity


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_condition = (city_data_df['Max Temp'] > 22) & (city_data_df['Max Temp'] < 35)
ideal_wind_condition = city_data_df['Wind Speed'] < 77
ideal_cloud_condition = city_data_df['Cloudiness'] < 55
ideal_weather_df = city_data_df[ideal_temp_condition & ideal_wind_condition & ideal_cloud_condition]


# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,daugavgriva,57.0432,24.0361,22.15,53,0,4.12,LV,1689488233
4,4,acarau,-2.8856,-40.1200,24.61,89,8,6.38,BR,1689488233
5,5,constantia,44.1833,28.6500,27.61,50,0,1.20,RO,1689488120
13,13,naze,28.3667,129.4833,29.51,74,9,7.01,JP,1689488254
16,16,ust-nera,64.5667,143.2000,23.81,45,52,2.40,RU,1689488273
...,...,...,...,...,...,...,...,...,...,...
540,540,dwarka,22.2394,68.9678,29.75,72,54,8.05,IN,1689489205
549,549,zaysan,47.4667,84.8667,32.39,16,26,7.02,KZ,1689489226
554,554,las khorey,11.1595,48.1967,34.15,47,27,2.01,SO,1689489226
555,555,inongo,-1.9500,18.2667,22.99,68,1,1.34,CD,1689489227


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,daugavgriva,LV,57.0432,24.0361,53,
4,acarau,BR,-2.8856,-40.1200,89,
5,constantia,RO,44.1833,28.6500,50,
13,naze,JP,28.3667,129.4833,74,
16,ust-nera,RU,64.5667,143.2000,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apikey": geoapify_key,
    "limit": 1,
    "radius": radius,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    time.sleep(3) # to not get rate limited
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
daugavgriva - nearest hotel: No hotel found
acarau - nearest hotel: No hotel found
constantia - nearest hotel: No hotel found
naze - nearest hotel: No hotel found
ust-nera - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
uturoa - nearest hotel: No hotel found
paracuru - nearest hotel: No hotel found
susuman - nearest hotel: No hotel found
mountain home - nearest hotel: No hotel found
upper grand lagoon - nearest hotel: No hotel found
shitanjing - nearest hotel: No hotel found
nar'yan-mar - nearest hotel: No hotel found
miri - nearest hotel: No hotel found
georgetown - nearest hotel: No hotel found


SSLError: HTTPSConnectionPool(host='api.geoapify.com', port=443): Max retries exceeded with url: /v2/places?apikey=2517eb982728425b848b4ed7f0a775be&limit=1&radius=10000&categories=accommodation.hotel&filter=circle%3A-5.1989%2C-35.4608%2C10000&bias=proximity%3A-35.4608%2C-5.1989 (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1007)')))

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points('Lng', 'Lat', 
                                  geo=True, 
                                  tiles='OSM',
                                  color='gray', 
                                  hover_cols=['Country', 'City', 'Hotel Name'],
                                  frame_height = 400, 
                                  frame_width = 750)

# Display the map
map_plot



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)

In [ ]:
# i keep getting rate limited but I was able to get the map plotted with
# different variables earlier while I was playing around with the 
# ideal weather condition variables
# so this map plot does not match what should be the output for the cells earlier 